In [33]:
--Selected the count of records, count of description, count of listing price, and count of people last visited the site
--Joined fiance and traffics sheet by the product id

Select     
    count(*) as total_rows, 
    count(description) as count_description,
    count(listing_price) as count_listing_price,
    count(last_visited) as count_last_visited
from
    info
inner join finance 
    using (product_id)
inner join traffic 
    using (product_id);
    


1 rows affected.


total_rows,count_description,count_listing_price,count_last_visited
3179,3117,3120,2928


In [35]:
--Selected the brad, listing price(converted from float to integer, and number of records
--Joined brands and fiance by product id
--Where the listing is higher than 0

Select 
    brands.brand, finance.listing_price::integer, count(*)
from
    brands 
inner join 
      finance
    using(product_id)
where 
    listing_price > 0
group by
    brand, listing_price  
order by 
    listing_price desc;
    


 * postgresql:///sports
77 rows affected.


brand,listing_price,count
Adidas,300,2
Adidas,280,4
Adidas,240,5
Adidas,230,8
Adidas,220,11
Nike,200,1
Adidas,200,8
Nike,190,2
Adidas,190,7
Nike,180,4


In [37]:
-- Selected  brand,  count of all products from finance, and total revenue
-- Created four fields based by price range as price_category
-- Joined byproduct_id and filter out products missing a value for brand

Select 
    brands.brand, 
    count(finance.product_id), 
    sum(finance.revenue) as total_revenue,
    Case 
        when listing_price < 42 then 'Budget'
        when listing_price >= 42 and listing_price < 74 then 'Average'
        when listing_price >= 74 and listing_price < 129 then 'Expensive'
        else 'Elite' End as price_category    
from
    brands
inner join finance
    using (product_id)
where 
    brands.brand is not null
group by
   brand, price_category
order by
    total_revenue desc;

 * postgresql:///sports
8 rows affected.


brand,count,total_revenue,price_category
Adidas,849,4626980.069999999,Expensive
Adidas,1060,3233661.060000001,Average
Adidas,307,3014316.8299999987,Elite
Adidas,359,651661.1200000002,Budget
Nike,357,595341.0199999992,Budget
Nike,82,128475.59000000003,Elite
Nike,90,71843.15000000004,Expensive
Nike,16,6623.5,Average


In [43]:
-- Calculated description_length
-- Convert rating to numeric and calculated average_rating
-- Joined byproduct_id 
-- Filtered products without missing values for description

select 
    trunc(length(i.description), -2) as description_length,
    round(avg(r.rating::numeric), 2) as average_rating
from info as i
inner join reviews as r 
    on i.product_id = r.product_id
where
    i.description is not null
group by
    description_length
order by
    description_length;

 * postgresql:///sports
7 rows affected.


description_length,average_rating
0,1.87
100,3.21
200,3.27
300,3.29
400,3.32
500,3.12
600,3.65


In [45]:
-- Selected brand,last_visited by month, and a count of all products in reviews aliased as num_reviews
-- Join traffic with reviews and brands on product_id
-- Group by brand and month, filtering out missing values for brand and month
-- Order the results by brand and month
select 
    b.brand, 
    date_part('month', t.last_visited) as month, 
    count(r.*) as num_reviews
from brands as b
inner join traffic as t 
    on b.product_id = t.product_id
inner join reviews as r 
    on t.product_id = r.product_id
group by
    b.brand, month
having
    b.brand is not null and 
    date_part('month', t.last_visited) is not null
ordered by 
    b.brand, month;

 * postgresql:///sports
24 rows affected.


brand,month,num_reviews
Adidas,1.0,253
Adidas,2.0,272
Adidas,3.0,269
Adidas,4.0,180
Adidas,5.0,172
Adidas,6.0,159
Adidas,7.0,170
Adidas,8.0,189
Adidas,9.0,181
Adidas,10.0,192
